In [1]:
%pip install pydicom
#%pip install pillow
%pip install numpy
%pip install matplotlib
%pip install scikit-image

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: C:\Users\User\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import pydicom
import numpy as np
from PIL import Image
import os
#import cv2
import matplotlib.pyplot as plt
from skimage import morphology
from scipy import ndimage

In [32]:
file_path = r'D:\Jaime_Andres\Univalle\TESIS ULTIMA OPORTUNIDAD\tesis_ultima_oportunidad\DataSet\prueba1\DATA_BASE\DATA_BASE_TEST\LMCI_TEST\LMCI\\'
def get_names(file_path):
    names = []
    for root, dirnames, filenames in os.walk(file_path):
        for filename in filenames:
            x , ext = os.path.splitext(filename)
            if ext in ['.dcm']:
                names.append(x)
    
    return names


In [33]:

def remove_noise_from_image(file_path, slice_num=0, display=False):
    medical_image = pydicom.read_file(file_path + name + '.dcm')
    image = medical_image.pixel_array
    shape = image.shape

    image_2d = image

    mask = image_2d <= 10
    selem = morphology.disk(2)

    segmentation = morphology.dilation(mask, selem)
    labels, label_nb = ndimage.label(segmentation)

    mask = labels == 0
    
    mask = morphology.dilation(mask, selem)
    mask = ndimage.morphology.binary_fill_holes(mask)
    mask = morphology.dilation(mask, selem)
    
    clean_image = mask * image_2d


    return clean_image

In [34]:
def add_pad(clean_image, new_height=256, new_width=256):
    height, width = clean_image.shape

    final_image = np.zeros((new_height, new_width))

    pad_left = int((new_width - width) / 2)
    pad_top = int((new_height - height) / 2)
    
    # Replace the pixels with the image's pixels
    final_image[pad_top:pad_top + height, pad_left:pad_left + width] = clean_image
    
    return final_image

In [35]:
def standardization(image):
    # Get brain mask
    mask = image == 0
    
    selem = morphology.disk(2)
    
    segmentation = morphology.dilation(mask, selem)
    labels, label_nb = ndimage.label(segmentation)

    mask = labels == 0
    
    mask = morphology.dilation(mask, selem)
    mask = ndimage.morphology.binary_fill_holes(mask)
    mask = morphology.dilation(mask, selem)
    
    # Normalize
    
    mean = image[mask].mean()
    std = image[mask].std()
    
    normalized_image = (image - mean) / std
    
    return normalized_image

In [36]:

new_path = r'D:\Jaime_Andres\Univalle\TESIS ULTIMA OPORTUNIDAD\tesis_ultima_oportunidad\DataSet\prueba1\DATA_BASE\DATA_BASE_TEST\IMAGES_PREPROCESSED_TEST\IMAGES\LMCI\\'
names = get_names(file_path)

for name in names:

    clean_image = remove_noise_from_image(file_path)

    final_image = add_pad(clean_image)
    
    normalized_image = standardization(final_image)

    preprocessed_image = (normalized_image - normalized_image.min()) * (255.0 / (normalized_image.max() - normalized_image.min()))
    preprocessed_image = preprocessed_image.astype('uint8')

    preprocessed_image = Image.fromarray(preprocessed_image)
    preprocessed_image.save(new_path + name + '.jpg')

C:\Users\javil\AppData\Local\Temp\ipykernel_18460\3884682074.py:17: DeprecationWarning: Please import `binary_fill_holes` from the `scipy.ndimage` namespace; the `scipy.ndimage.morphology` namespace is deprecated and will be removed in SciPy 2.0.0.
  mask = ndimage.morphology.binary_fill_holes(mask)
C:\Users\javil\AppData\Local\Temp\ipykernel_18460\3186814896.py:13: DeprecationWarning: Please import `binary_fill_holes` from the `scipy.ndimage` namespace; the `scipy.ndimage.morphology` namespace is deprecated and will be removed in SciPy 2.0.0.
  mask = ndimage.morphology.binary_fill_holes(mask)
C:\Users\javil\AppData\Local\Temp\ipykernel_18460\3186814896.py:18: RuntimeWarning: Mean of empty slice.
  mean = image[mask].mean()
C:\Users\javil\AppData\Local\Temp\ipykernel_18460\2628064472.py:13: RuntimeWarning: invalid value encountered in cast
  preprocessed_image = preprocessed_image.astype('uint8')
